In [1]:
## add ignore warnings for now, will remove and debug once full algorithm is complete
import warnings
warnings.filterwarnings("ignore")

## import packages/libraries
from time import perf_counter, clock_gettime_ns, CLOCK_REALTIME
import numpy as np
import pandas as pd
from multiprocessing import Pool, cpu_count
import sys
import sqlite3

## append filepath to allow files to be called from within project folder
sys.path.append('/home/gerard/Desktop/capstone_project/patoms')
sys.path.append('/home/gerard/Desktop/capstone_project')

## call locally created functions
from snapshot_2d_pattern_v6 import patoms2d
from snapshot_3d_pattern_v6 import patoms3d
from pattern_2d_compare_v5 import pattern_compare_2d
from pattern_3d_compare_v4 import pattern_compare_3d
from updating_ref_table_v0 import update_ref_table

In [2]:
## connect to 2d database
con2d = sqlite3.connect("ref2d_v4.db")
cur2d = con2d.cursor()

In [3]:
######## DONT TOUCH ##########

# ## read 2d ref database into memory and convert to ref patoms for comparison
# working_ref_patoms = []
# con2dref = sqlite3.connect("ref2d_v4.db")
# cur2dref = con2dref.cursor()
# ref_names = [name for (name,) in cur2dref.execute("select name from sqlite_master where type='table' and name like '%ref%';").fetchall()]
# for i in ref_names:
#     table = cur2dref.execute(f"select * from {i};").fetchall()
#     table_array = np.array(table)
#     working_ref_patoms.append(table_array)

# ref_patoms_array = np.vstack(working_ref_patoms[:50])
# ref_indices = np.unique(ref_patoms_array[:,6:8],axis=0)
# print(max(ref_names))


In [ ]:
######## DONT TOUCH ##########

# one_sec_time = perf_counter()
# val = 0
# while val <= 0:
#     rand_array = np.random.random((1, 720, 1280)) # this needs to expand to include acceleromter and audio data
#     y_len = rand_array.shape[1]
#     x_len = rand_array.shape[2]
#     # ingest data frame by frame
#     for frame in range(rand_array.shape[0]):
#         ####################### FIRST TASK: FIND PATTERNS IN FRAME ######################
#         frame_patoms = patoms2d(x_len, y_len, rand_array[frame,:,:], frame)
#         # output: [norm_x, norm_y, pattern_centroid_x, pattern_centroid_y, quad, quad_cnt, patom_ind, frame_ind_arr, patom_time]
#         num_patoms = len(frame_patoms)
#         val += 1
#         ############## SECOND TASK: COMPARE NEW PATOMS AGAINST REF PATOMS ###############
#         atime = perf_counter()
#         with Pool(processes=8) as pool:
#             items = [(frame_patoms[i], ref_patoms_array) for i in range(num_patoms)]
#             comp_results = pool.starmap(pattern_compare_2d, items) #e.g. ['px','py','pxc','pyc','pq','pqlen','ppind','pfind','ptime','rpind','rfind','xc_d','yc_d','x_d','y_d','similar']
#             print("Time to compare 2D patterns with multiprocessing (secs):", (perf_counter()-atime))
#             ## loop through the output of the comparison function
#             witime = perf_counter()
#             for ix, i in enumerate(comp_results):
#                 # get max table number from 2d database
#                 next_table_num = int([table for (table,) in cur2d.execute("SELECT name FROM sqlite_master WHERE type='table';").fetchall()].pop(-1)[-6:]) + 1 
#                 next_table_num = str(next_table_num).zfill(6)
#                 # get existing table names
#                 existing_tables = [names for (names,) in cur2d.execute("select name from sqlite_master where type='table';").fetchall()]
#                 # get patom from this iteration
#                 columns = np.array([0,1,2,3,4,5,6,7,8,11,12,13,14], dtype=np.intp)
#                 rows = np.array(range(np.unique(i[:,:1],axis=0).shape[0]), dtype=np.intp)
#                 patom = i[np.ix_(rows,columns)]
#                 # extract reference tables that are similar to the patom in the current iteration
#                 similar_patoms = i[i[:,15] == 1]
#                 if similar_patoms.shape[0] > 0:
#                     ref_patom_indices = similar_patoms[:,9:11] 
#                     ref_table_name_indices = [xs for x in np.argwhere(ref_indices[np.isin(ref_indices, ref_patom_indices)]).tolist() for xs in x]
#                     if ref_table_name_indices: # why are there empty lists??? there are empty lsts as there are ref indices that do not have any match to new similar patoms
#                         ref_tables_numbers = [ref_names[x][-6:] for x in ref_table_name_indices]
#                     for j in ref_tables_numbers:
#                         cur2d.executemany(f"INSERT INTO pat_2d_{j}(x_pos_dist, y_pos_dist, x_cent, y_cent, quad, quad_cnt, patom_ind, frame_ind, patom_time, \
#                                           xc_d, yc_d, x_d, y_d) VALUES (?,?,?,?,?,?,?,?,?,?,?,?,?)", patom)
#                 # if no matching ref patoms, write new patom to new table
#                 non_similar_patoms = i[i[:,15] == 0]
#                 if non_similar_patoms.shape[0] > 0:
#                     cur2d.execute(f"CREATE TABLE pat_2d_{next_table_num}(x_pos_dist, y_pos_dist, x_cent, y_cent, quad, quad_cnt, patom_ind, frame_ind, patom_time, \
#                                   xc_d, yc_d, x_d, y_d);")
#                     cur2d.executemany(f"INSERT INTO pat_2d_{next_table_num}(x_pos_dist, y_pos_dist, x_cent, y_cent, quad, quad_cnt, patom_ind, frame_ind, patom_time, \
#                                       xc_d, yc_d, x_d, y_d) VALUES (?,?,?,?,?,?,?,?,?,?,?,?,?)", patom)    
                
#                 con2d.commit()
#             ## major bottleneck is the insert/create steps
#             print("Time to write/insert patoms to existing/new tables in database (secs):", (perf_counter()-witime))

# print("Time to process 1 second of data against 50 reference patoms(secs):", (perf_counter()-one_sec_time))
# con2d.close()
# con2dref.close()

In [6]:
########################################################################################################
############## next steps is reconcile ref patoms and put time order on patom sequence #################
########################################################################################################

# fileds in tables: [x_pos_dist, y_pos_dist, x_cent, y_cent, quad, quad_cnt, patom_ind, frame_ind, patom_time, xc_d, yc_d, x_d, y_d]
working_tables = []
with open('/home/gerard/Desktop/capstone_project/ref2d_v4.db', 'rb') as f:
    con2d = sqlite3.connect(":memory:")
    con2d.deserialize(f.read())
    cur2d = con2d.cursor()
    nonref_names = [name for (name,) in cur2d.execute("select name from sqlite_master where type='table' nd name not like '%ref%';").fetchall()]
    for i in nonref_names:
        row = cur2d.execute(f"select * from {i} LIMIT 1;").fetchone()
        #table_array = np.array(table)
        if row is None:
            pass
        else:
            table = cur2d.execute(f"select * from {i};").fetchall()
            working_tables.append(table)
            # within table 